# Price Prediction Model Data Prep

### Import dependencies

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

import plotly.express as px

try:
  import statsmodels
except:
  !pip install statsmodels
  import statsmodels

MAX_AVG_PRICE_LIMIT = 1500

### Specify the city to process

In [ ]:
CITY_NAME = 'Nashville'   

### Dictionary of file names and locations

In [ ]:
city_data_dict = {
    'Nashville': {
        'filepath': '../../data/',
        'input_filename_calendar': 'raw/AirBnB/Samples/calendar.csv',
        'input_filename_listings': 'raw/AirBnB/Samples/listings.csv',
        'input_filename_amenities': 'processed/OHE_amenities_nashville.csv', 
        'input_filename_attractions': 'processed/nearby_entertainment_nashville.csv', 
        'output_filename': 'processed/train_test_data_nashville.csv',
        },
}

### Set the file names for the chosen city

In [ ]:
path_to_use = city_data_dict[CITY_NAME]['filepath']
input_filename_calendar = city_data_dict[CITY_NAME]['input_filename_calendar']
input_filename_listings = city_data_dict[CITY_NAME]['input_filename_listings']
input_filename_amenities = city_data_dict[CITY_NAME]['input_filename_amenities']
input_filename_attractions = city_data_dict[CITY_NAME]['input_filename_attractions']
output_filename = city_data_dict[CITY_NAME]['output_filename']

## Read and Clean the Calendar Data

In [ ]:
calendar_df = pd.read_csv(path_to_use+input_filename_calendar)
calendar_df = calendar_df.dropna()
calendar_df['price'] = calendar_df['price'].replace({'\$': '', ',': ''},regex=True).astype('float') # https://pbpython.com/currency-cleanup.html
calendar_df['adjusted_price'] = calendar_df['adjusted_price'].replace({'\$': '', ',': ''},regex=True).astype('float')
print(calendar_df.shape)
calendar_df.head()

(2965287, 7)


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,6422,2022-12-22,f,40.0,40.0,30.0,365.0
1,6422,2022-12-23,f,40.0,40.0,30.0,365.0
2,6422,2022-12-24,f,40.0,40.0,30.0,365.0
3,6422,2022-12-25,f,40.0,40.0,30.0,365.0
4,6422,2022-12-26,f,40.0,40.0,30.0,365.0


In [ ]:
calendar_df.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

## Read and Clean the Listings Data

In [ ]:
listings_df = pd.read_csv(path_to_use+input_filename_listings, dtype={'price': str})
listings_df['price'] = listings_df['price'].replace({'\$': '', ',': ''},regex=True).astype('float')
print(listings_df.shape)
listings_df.head()

(8127, 75)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6422,https://www.airbnb.com/rooms/6422,20221221170309,2022-12-22,city scrape,Nashville Charm,30 day or more rental during COVID. Show COVID...,Historic East Nashville is home to many new an...,https://a0.muscache.com/pictures/pro_photo_too...,12172,...,4.96,4.92,4.98,NaN,f,1,0,1,0,4.06
1,59576,https://www.airbnb.com/rooms/59576,20221221170309,2022-12-21,city scrape,Large Main Suite near Lake *ladies only NS plz,non-smokers only please<br /><br />furnished r...,We live on a cul-de-sac in a small community o...,https://a0.muscache.com/pictures/4e4e34db-3155...,812128,...,5.00,5.00,5.00,NaN,f,10,1,9,0,0.34
2,72906,https://www.airbnb.com/rooms/72906,20221221170309,2022-12-21,city scrape,Vandy/Belmont/10 mins to Broadway - Sunny 800 ...,Entire top floor. Private. 800 sq ft of bright...,Historic Belmont-Hillsboro neighborhood. Walk-...,https://a0.muscache.com/pictures/58602855/3788...,176117,...,4.99,4.96,4.89,NaN,f,1,1,0,0,4.76
3,319705,https://www.airbnb.com/rooms/319705,20221221170309,2022-12-21,city scrape,"SuperSweetSTUDIO, jacuzzi, open Nov 23, 6 mo",Huge fully furnished private room /studio apar...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/3470453/c7d82...,22296,...,4.68,4.62,4.57,NaN,f,8,0,7,1,0.36
4,289242,https://www.airbnb.com/rooms/289242,20221221170309,2022-12-21,city scrape,"MorningstarHouse, monthly room- open Aug 19",Morningstar House! Extended stay - price reduc...,The Morningstar House is in a quiet cul-de-sac...,https://a0.muscache.com/pictures/40a7ab72-20f7...,22296,...,4.78,4.47,4.69,NaN,f,8,0,7,1,0.57


In [ ]:
listings_df.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,8.127000e+03,8.127000e+03,8.127000e+03,8127.000000,8127.000000,0.0,8127.000000,8127.000000,8127.000000,0.0,...,7101.000000,7102.000000,7101.000000,7101.000000,0.0,8127.000000,8127.000000,8127.000000,8127.000000,7114.000000
mean,2.275005e+17,2.022122e+13,1.587349e+08,95.412821,161.255199,NaN,36.164076,-86.767898,6.642057,NaN,...,4.869330,4.871018,4.777260,4.753404,NaN,25.439523,23.366802,1.919528,0.006398,2.366975
std,3.195781e+17,0.000000e+00,1.519691e+08,359.347385,726.329515,NaN,0.041382,0.051728,3.780720,NaN,...,0.306587,0.315933,0.319008,0.333151,NaN,44.680552,44.278399,8.361461,0.103868,2.501614
min,6.422000e+03,2.022122e+13,1.147000e+03,1.000000,1.000000,NaN,35.983300,-87.035210,0.000000,NaN,...,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.010000
25%,3.176405e+07,2.022122e+13,2.947057e+07,1.000000,2.000000,NaN,36.144645,-86.795780,4.000000,NaN,...,4.870000,4.880000,4.720000,4.700000,NaN,1.000000,1.000000,0.000000,0.000000,0.810000
50%,4.914556e+07,2.022122e+13,1.014269e+08,5.000000,7.000000,NaN,36.159830,-86.775414,6.000000,NaN,...,4.960000,4.970000,4.880000,4.830000,NaN,4.000000,3.000000,0.000000,0.000000,1.910000
75%,6.101710e+17,2.022122e+13,2.635022e+08,41.000000,54.000000,NaN,36.186470,-86.742985,10.000000,NaN,...,5.000000,5.000000,4.970000,4.920000,NaN,26.000000,19.000000,0.000000,0.000000,3.270000
max,7.855243e+17,2.022122e+13,4.912941e+08,2958.000000,7361.000000,NaN,36.398480,-86.546060,16.000000,NaN,...,5.000000,5.000000,5.000000,5.000000,NaN,183.000000,183.000000,57.000000,2.000000,100.350000


In [ ]:
listings_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

### Convert bathroom text to numeric values

In [ ]:
# replace [baths, bath, private, shared, half, -, space] with ''; convert to float, fill blanks with zero
replace_dict = {
    'baths': '',
    'bath': '',
    'private': '',
    'shared':'',
    'half':'',
    '-':'',
    ' ':''
}
listings_df['num_bathrooms'] = listings_df['bathrooms_text'].str.lower()
listings_df['num_bathrooms'] = listings_df['num_bathrooms'].replace(replace_dict,regex=True)
listings_df['num_bathrooms'] = np.where(listings_df['num_bathrooms']=='', 0, listings_df['num_bathrooms']).astype('float')
listings_df['num_bathrooms'].value_counts(normalize=True).head(10)

1.0    0.451502
2.0    0.169375
2.5    0.086780
3.5    0.078656
3.0    0.068932
4.0    0.058099
1.5    0.036805
4.5    0.014771
6.0    0.006524
8.0    0.006278
Name: num_bathrooms, dtype: float64

### Convert t/f values to 1/0 indicators

In [ ]:
indidcator_column_dict = {
    # t/f column name: indicator column name
    'has_availability': 'availability_ind',
    'host_is_superhost': 'host_is_superhost_ind',
    'host_has_profile_pic': 'host_profile_pic_ind',
    'host_identity_verified': 'host_identity_verified_ind'
}
for key, val in indidcator_column_dict.items():
    listings_df[val] = np.where(listings_df[key]=='t', 1, 0)
    print(listings_df[val].value_counts(normalize=True))


1    0.981666
0    0.018334
Name: availability_ind, dtype: float64
1    0.522333
0    0.477667
Name: host_is_superhost_ind, dtype: float64
1    0.996801
0    0.003199
Name: host_profile_pic_ind, dtype: float64
1    0.85542
0    0.14458
Name: host_identity_verified_ind, dtype: float64


### Calculate the average price per listing

In [ ]:
avg_price_per_listing_df = calendar_df[['listing_id', 'price']].groupby('listing_id').mean()
avg_price_per_listing_df = avg_price_per_listing_df.loc[avg_price_per_listing_df['price'] <= MAX_AVG_PRICE_LIMIT, :]
print(avg_price_per_listing_df.shape)
avg_price_per_listing_df.head()

(7965, 1)


,price
listing_id,
6422,40.000000
59576,45.000000
72906,104.616438
258817,39.000000
289242,33.000000


In [ ]:
cols_to_keep = ['id', 'accommodates', 'num_bathrooms', 'bedrooms', 'beds', 
                'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 
                'availability_ind', 'host_is_superhost_ind', 
                'latitude', 'longitude']
filtered_listings_df = listings_df.loc[listings_df['room_type']=='Entire home/apt', cols_to_keep]
filtered_listings_df = filtered_listings_df.dropna()
filtered_listings_df.set_index('id', inplace=True)

In [ ]:
print(listings_df.shape, filtered_listings_df.shape)

(8127, 80) (6875, 10)


## Read the Amenities Data

In [ ]:
remapped_amentities = {'Free Parking': ['Free Parking'],
    'Kitchen Appliances': ['Coffee Maker','Refrigerator','Oven','Stove','Dishwasher','Freezer','Toaster','Blender'],
    'Patio or Balcony': ['Patio or Balcony'],
    'WiFi': ['WiFi'],
    'Smoke Alarm': ['Smoke Alarm'],
    'Heating and Cooling': ['Air Conditioning', 'Heating'],
    'TV': ['TV'],
    'Essentials': ['Essentials'],
    'Kitchen': ['Kitchen'],
    'Hair Dryer': ['Hair Dryer'],
    'Long Term Stays Allowed': ['Long Term Stays Allowed'],
    'Laundry Essentials': ['Iron', 'Hangers', 'Washer', 'Dryer'],
    'Toiletries': ['Shampoo', 'Body Soap', 'Conditioner'],
    'Kitchen Essentials': ['Diningware','Microwave','Cooking Basics','Dining Table'],
    'Hot Water': ['Hot Water'],
    'Fire Extinguisher': ['Fire Extinguisher'],
    'Carbon Monoxide Alarm': ['Carbon Monoxide Alarm'],
    'Bed Linens': ['Bed Linens'],
    'Self Check-in': ['Self Check-in'],
    'Private Entrance': ['Private Entrance'],
    'First Aid Kit': ['First Aid Kit'],
    'Extra Pillows and Blankets': ['Extra Pillows and Blankets'],
    'Dedicated Workspace': ['Dedicated Workspace'],
    'Surveillance': ['Surveillance'],
    'Backyard': ['Backyard'],
    'Clothing Storage': ['Clothing Storage'],
    'Wine Glasses': ['Wine Glasses'],
    'Cleaning Products': ['Cleaning Products'],
    'Keypad': ['Keypad'],
    'BBQ': ['BBQ'],
    'Shades': ['Shades'],
    'Luggage Dropoff Allowed': ['Luggage Dropoff Allowed'],
    'Smart Lock': ['Smart Lock'],
    'Pets Allowed': ['Pets Allowed'],
    'Baby Essentials': ['Baby Crib', 'High Chair'],
    'Pool': ['Pool'],
    'Gym': ['Gym'],
    'Fire Pit': ['Fire Pit'],
    'Elevator': ['Elevator'],
    'Lockbox': ['Lockbox'],
    'Fireplace': ['Fireplace'],
    'Paid Parking': ['Paid Parking'],
    'Laundromat Nearby': ['Laundromat Nearby'],
    'Exercise Equipment': ['Exercise Equipment'],
    'View': ['City Skyline View','Garden View','Courtyard View','River View','Pool View','Park View','Resort View',
    'Mountain View','Valley View','Lake View','Golf Course View','Marina View','Canal View','Bay View','Vineyard View'],
    'Hot Tub': ['Hot Tub'],
    'Toys': ['Toys'],
    'Sound System': ['Sound System'],
    'Beach Access': ['Beach Access'],
    'EV Charger': ['EV Charger']
    }

In [ ]:
def read_and_process_amenities_data(min_variation):
    df = pd.read_csv(path_to_use+input_filename_amenities, index_col='id')

    # remap related amentiy columns
    for new_col, old_cols in remapped_amentities.items():
        col_vals = [df[old_col] for old_col in old_cols]
        col_vals_arr = np.array(col_vals)
        new_col_vals = col_vals_arr.max(axis=0)
        df = df.drop(columns=old_cols)
        df[new_col] = new_col_vals
   
    return_dict = dict()
    amenities = []
    col_means = []
    for col in df.columns:
        col_mean = np.mean(df[col])
        amenities.append(col)
        col_means.append(col_mean)

    col_means_df = pd.DataFrame({'mean_value': col_means}, index=amenities)
    col_means_df.index.name = 'amenity'

    row_criteria = (col_means_df['mean_value'] >= min_variation) & (
                    col_means_df['mean_value'] <= (1-min_variation))
    amenities_kept = list(col_means_df.loc[row_criteria, :].index)
    amenities_not_kept = list(col_means_df.loc[~(row_criteria), :].index)
    return_dict['final_amenities_df'] = df[amenities_kept]
    return_dict['amenities_kept'] = amenities_kept
    return_dict['filtered_out_amenities_df'] = df[amenities_not_kept]
    return_dict['amenities_not_kept'] = amenities_not_kept
    
    return return_dict

In [ ]:
min_variation = 0.1
amenities_df = None
if input_filename_amenities != '':
    amenities_dict = read_and_process_amenities_data(min_variation)
    amenities_df = amenities_dict['final_amenities_df']
    amenities_kept = amenities_dict['amenities_kept']
    amenities_not_kept = amenities_dict['amenities_not_kept']
    print('final shape:', amenities_df.shape)
    print('kept:', len(amenities_kept), amenities_kept)
    print('not kept:', len(amenities_not_kept), amenities_not_kept)

final shape: (8127, 39)
kept: 39 ['Free Parking', 'Kitchen Appliances', 'Patio or Balcony', 'Kitchen', 'Hair Dryer', 'Long Term Stays Allowed', 'Toiletries', 'Kitchen Essentials', 'Hot Water', 'Fire Extinguisher', 'Carbon Monoxide Alarm', 'Bed Linens', 'Self Check-in', 'Private Entrance', 'First Aid Kit', 'Extra Pillows and Blankets', 'Dedicated Workspace', 'Surveillance', 'Backyard', 'Clothing Storage', 'Wine Glasses', 'Cleaning Products', 'Keypad', 'BBQ', 'Shades', 'Luggage Dropoff Allowed', 'Smart Lock', 'Pets Allowed', 'Baby Essentials', 'Pool', 'Gym', 'Fire Pit', 'Elevator', 'Lockbox', 'Fireplace', 'Paid Parking', 'Laundromat Nearby', 'Exercise Equipment', 'View']
not kept: 11 ['WiFi', 'Smoke Alarm', 'Heating and Cooling', 'TV', 'Essentials', 'Laundry Essentials', 'Hot Tub', 'Toys', 'Sound System', 'Beach Access', 'EV Charger']


['WiFi', 'Smoke Alarm', 'Heating and Cooling', 'TV', 'Essentials', 'Laundry Essentials', 'Hot Tub', 'Toys', 'Sound System', 'Beach Access', 'EV Charger']


In [ ]:
if input_filename_amenities != '':
    print(amenities_df.head())

          Free Parking  Kitchen Appliances  Patio or Balcony  Kitchen  \
id                                                                      
10017234             1                   1                 1        1   
10036680             1                   1                 0        1   
10056974             1                   1                 1        1   
1006989              1                   1                 1        0   
1009550              1                   1                 1        1   

          Hair Dryer  Long Term Stays Allowed  Toiletries  Kitchen Essentials  \
id                                                                              
10017234           1                        1           1                   1   
10036680           1                        1           1                   1   
10056974           1                        1           1                   1   
1006989            1                        0           1                   0   
10

In [ ]:
if input_filename_amenities != '':
    print(amenities_df.columns)

Index(['Free Parking', 'Kitchen Appliances', 'Patio or Balcony', 'Kitchen',
       'Hair Dryer', 'Long Term Stays Allowed', 'Toiletries',
       'Kitchen Essentials', 'Hot Water', 'Fire Extinguisher',
       'Carbon Monoxide Alarm', 'Bed Linens', 'Self Check-in',
       'Private Entrance', 'First Aid Kit', 'Extra Pillows and Blankets',
       'Dedicated Workspace', 'Surveillance', 'Backyard', 'Clothing Storage',
       'Wine Glasses', 'Cleaning Products', 'Keypad', 'BBQ', 'Shades',
       'Luggage Dropoff Allowed', 'Smart Lock', 'Pets Allowed',
       'Baby Essentials', 'Pool', 'Gym', 'Fire Pit', 'Elevator', 'Lockbox',
       'Fireplace', 'Paid Parking', 'Laundromat Nearby', 'Exercise Equipment',
       'View'],
      dtype='object')


## Read the Attractions Data

In [ ]:
attractions_df = None
if input_filename_attractions != '':
    attractions_df = pd.read_csv(path_to_use+input_filename_attractions, 
                                    index_col='AirBnB_id', usecols=range(1,20))
    attractions_df.index.name = 'id'
    print(attractions_df.shape)

(8127, 18)


In [ ]:
if input_filename_attractions != '':
    print(attractions_df.head())

          active_within_0_2  arts_within_0_2  food_within_0_2  \
id                                                              
10017234                  1                0                1   
10036680                  0                1                0   
10056974                  0                0                0   
1006989                   0                0                2   
1009550                   0                1                0   

          nightlife_within_0_2  restaurants_within_0_2  shopping_within_0_2  \
id                                                                            
10017234                     0                       8                    0   
10036680                     1                       4                    0   
10056974                     0                       0                    0   
1006989                      2                       8                    0   
1009550                      1                       3                

In [ ]:
if input_filename_attractions != '':
    print(attractions_df.columns)

Index(['active_within_0_2', 'arts_within_0_2', 'food_within_0_2',
       'nightlife_within_0_2', 'restaurants_within_0_2', 'shopping_within_0_2',
       'active_within_2_4', 'arts_within_2_4', 'food_within_2_4',
       'nightlife_within_2_4', 'restaurants_within_2_4', 'shopping_within_2_4',
       'active_beyond_4', 'arts_beyond_4', 'food_beyond_4',
       'nightlife_beyond_4', 'restaurants_beyond_4', 'shopping_beyond_4'],
      dtype='object')


## Merge the Calendar, Listings, Amenities, and Attractions Data

In [ ]:
if input_filename_amenities == '':
    merged_df = filtered_listings_df
else:
    merged_df = pd.merge(filtered_listings_df, amenities_df, left_index=True, right_index=True)

if input_filename_attractions == '':
    pass 
else:
    merged_df = pd.merge(merged_df, attractions_df, left_index=True, right_index=True)

print(merged_df.shape)
merged_df.head()

(6875, 67)


,accommodates,num_bathrooms,bedrooms,beds,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_ind,host_is_superhost_ind,latitude,longitude,...,food_within_2_4,nightlife_within_2_4,restaurants_within_2_4,shopping_within_2_4,active_beyond_4,arts_beyond_4,food_beyond_4,nightlife_beyond_4,restaurants_beyond_4,shopping_beyond_4
id,,,,,,,,,,,,,,,,,,,,,
72906,4,1.0,2.0,2.0,2.0,1125.0,1,1,36.13122,-86.80066,...,0,1,3,0,0,0,0,0,0,0
431258,4,2.5,2.0,2.0,2.3,1101.9,1,1,36.17580,-86.79950,...,0,2,3,0,0,0,0,0,0,0
329997,2,1.0,1.0,1.0,2.2,1086.4,1,1,36.17580,-86.79950,...,0,2,3,0,0,0,0,0,0,0
1885504,6,2.0,2.0,3.0,2.6,1125.0,1,0,36.10963,-86.74195,...,2,4,8,0,0,0,0,1,2,0
632636,2,1.5,1.0,1.0,2.3,60.0,1,1,36.17230,-86.79250,...,0,2,3,0,0,0,0,0,0,0


In [ ]:
merged_df = pd.merge(merged_df, avg_price_per_listing_df, left_index=True, right_index=True)
print(merged_df.shape)
merged_df.head()

(6738, 68)


,accommodates,num_bathrooms,bedrooms,beds,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_ind,host_is_superhost_ind,latitude,longitude,...,nightlife_within_2_4,restaurants_within_2_4,shopping_within_2_4,active_beyond_4,arts_beyond_4,food_beyond_4,nightlife_beyond_4,restaurants_beyond_4,shopping_beyond_4,price
72906,4,1.0,2.0,2.0,2.0,1125.0,1,1,36.13122,-86.80066,...,1,3,0,0,0,0,0,0,0,104.616438
431258,4,2.5,2.0,2.0,2.3,1101.9,1,1,36.17580,-86.79950,...,2,3,0,0,0,0,0,0,0,351.986301
329997,2,1.0,1.0,1.0,2.2,1086.4,1,1,36.17580,-86.79950,...,2,3,0,0,0,0,0,0,0,127.887671
1885504,6,2.0,2.0,3.0,2.6,1125.0,1,0,36.10963,-86.74195,...,4,8,0,0,0,0,1,2,0,133.876712
632636,2,1.5,1.0,1.0,2.3,60.0,1,1,36.17230,-86.79250,...,2,3,0,0,0,0,0,0,0,163.739726


In [ ]:
cols = list(merged_df.columns)[:50]
merged_df[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6738 entries, 72906 to 760743300133591095
Data columns (total 50 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   accommodates                6738 non-null   int64  
 1   num_bathrooms               6738 non-null   float64
 2   bedrooms                    6738 non-null   float64
 3   beds                        6738 non-null   float64
 4   minimum_nights_avg_ntm      6738 non-null   float64
 5   maximum_nights_avg_ntm      6738 non-null   float64
 6   availability_ind            6738 non-null   int64  
 7   host_is_superhost_ind       6738 non-null   int64  
 8   latitude                    6738 non-null   float64
 9   longitude                   6738 non-null   float64
 10  Free Parking                6738 non-null   int64  
 11  Kitchen Appliances          6738 non-null   int64  
 12  Patio or Balcony            6738 non-null   int64  
 13  Kitchen        

In [ ]:
cols = list(merged_df.columns)[50:]
merged_df[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6738 entries, 72906 to 760743300133591095
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   arts_within_0_2         6738 non-null   int64  
 1   food_within_0_2         6738 non-null   int64  
 2   nightlife_within_0_2    6738 non-null   int64  
 3   restaurants_within_0_2  6738 non-null   int64  
 4   shopping_within_0_2     6738 non-null   int64  
 5   active_within_2_4       6738 non-null   int64  
 6   arts_within_2_4         6738 non-null   int64  
 7   food_within_2_4         6738 non-null   int64  
 8   nightlife_within_2_4    6738 non-null   int64  
 9   restaurants_within_2_4  6738 non-null   int64  
 10  shopping_within_2_4     6738 non-null   int64  
 11  active_beyond_4         6738 non-null   int64  
 12  arts_beyond_4           6738 non-null   int64  
 13  food_beyond_4           6738 non-null   int64  
 14  nightlife_beyond_4    

# Exploratory Data Analysis

In [ ]:
eda_df = merged_df.copy()
eda_df.columns

Index(['accommodates', 'num_bathrooms', 'bedrooms', 'beds',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_ind',
       'host_is_superhost_ind', 'latitude', 'longitude', 'Free Parking',
       'Kitchen Appliances', 'Patio or Balcony', 'Kitchen', 'Hair Dryer',
       'Long Term Stays Allowed', 'Toiletries', 'Kitchen Essentials',
       'Hot Water', 'Fire Extinguisher', 'Carbon Monoxide Alarm', 'Bed Linens',
       'Self Check-in', 'Private Entrance', 'First Aid Kit',
       'Extra Pillows and Blankets', 'Dedicated Workspace', 'Surveillance',
       'Backyard', 'Clothing Storage', 'Wine Glasses', 'Cleaning Products',
       'Keypad', 'BBQ', 'Shades', 'Luggage Dropoff Allowed', 'Smart Lock',
       'Pets Allowed', 'Baby Essentials', 'Pool', 'Gym', 'Fire Pit',
       'Elevator', 'Lockbox', 'Fireplace', 'Paid Parking', 'Laundromat Nearby',
       'Exercise Equipment', 'View', 'active_within_0_2', 'arts_within_0_2',
       'food_within_0_2', 'nightlife_within_0_2', 

## Review Stats for Each Feature

In [ ]:
eda_df.describe()

,accommodates,num_bathrooms,bedrooms,beds,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_ind,host_is_superhost_ind,latitude,longitude,...,nightlife_within_2_4,restaurants_within_2_4,shopping_within_2_4,active_beyond_4,arts_beyond_4,food_beyond_4,nightlife_beyond_4,restaurants_beyond_4,shopping_beyond_4,price
count,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,...,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000,6738.000000
mean,7.170377,2.169635,2.504601,4.368507,7.059439,739.067171,0.987088,0.548234,36.164769,-86.770129,...,1.484417,4.943455,0.006827,0.011131,0.027308,0.158801,0.493915,2.046008,0.007124,341.368923
std,3.528261,1.329220,1.412552,3.671092,25.153461,453.315256,0.112903,0.497705,0.039802,0.049016,...,1.584251,4.114934,0.082349,0.175781,0.165700,0.609908,1.087547,3.673472,0.114068,235.542074
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,35.983300,-87.035210,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
25%,4.000000,1.000000,1.000000,2.000000,2.000000,365.000000,1.000000,0.000000,36.145000,-86.796290,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,169.967123
50%,6.000000,2.000000,2.000000,3.000000,2.200000,1125.000000,1.000000,1.000000,36.161475,-86.775710,...,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,278.989041
75%,10.000000,3.000000,3.000000,6.000000,3.000000,1125.000000,1.000000,1.000000,36.186467,-86.746303,...,2.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,446.531507
max,16.000000,16.000000,16.000000,111.000000,845.500000,1500.000000,1.000000,1.000000,36.398480,-86.546060,...,8.000000,18.000000,1.000000,5.000000,2.000000,10.000000,7.000000,19.000000,3.000000,1500.000000


## Look at the distribution of price

In [ ]:
fig = px.histogram(eda_df, x='price')
fig.show()

## Look at price by size-related features

In [ ]:
fig = px.scatter(eda_df, x='accommodates', y='price', 
                        marginal_x='histogram', marginal_y='histogram', trendline='ols')
fig.show()

In [ ]:
fig = px.scatter(eda_df, x='num_bathrooms', y='price', 
                marginal_x='histogram', marginal_y='histogram', trendline='ols')
fig.show()

In [ ]:
fig = px.scatter(eda_df.sample(500), x='num_bathrooms', y='accommodates',
                marginal_x='histogram', marginal_y='histogram', trendline='ols', )
fig.show()

## Look at the frequency of the most common values in size-related columns

In [ ]:
for col in ['accommodates', 'num_bathrooms', 'bedrooms', 'beds']:
    print(eda_df[col].value_counts(normalize=True)[:6])

4     0.197982
6     0.183437
12    0.141437
10    0.125705
8     0.121104
2     0.079400
Name: accommodates, dtype: float64
1.0    0.387801
2.0    0.194420
2.5    0.102256
3.5    0.092757
3.0    0.079994
4.0    0.065450
1.5    0.031909
4.5    0.017067
6.0    0.006382
Name: num_bathrooms, dtype: float64
1.0    0.285099
2.0    0.252152
3.0    0.233304
4.0    0.195162
5.0    0.010389
8.0    0.010240
6.0    0.009350
Name: bedrooms, dtype: float64
2.0    0.224547
3.0    0.166073
4.0    0.137281
1.0    0.122440
5.0    0.090531
6.0    0.079549
Name: beds, dtype: float64


In [ ]:
fig = px.scatter(eda_df, x='accommodates', y='price', 
                marginal_x='histogram', marginal_y='histogram',
                facet_col='host_is_superhost_ind', trendline='ols') # trendline requires statsmodels
fig.show()

In [ ]:
fig = px.scatter(eda_df, x='bedrooms', y='price', 
                marginal_x='histogram', marginal_y='histogram',
                facet_col='host_is_superhost_ind', trendline='ols') # trendline requires statsmodels
fig.show()

In [ ]:
cols = ['host_is_superhost_ind', 'accommodates', 'bedrooms', 'num_bathrooms']
eda_df_grp_by_sh = eda_df[cols].groupby('host_is_superhost_ind').mean()
eda_df_grp_by_sh

,accommodates,bedrooms,num_bathrooms
host_is_superhost_ind,,,
0,6.674442,2.388305,2.032852
1,7.579047,2.600433,2.282350


Observation: Super-hosts are associated with larger properties and price is more sensitive to size-related features for super-hosts than for non-supers.

## Look at price by location

In [ ]:
fig = px.scatter(eda_df, x='latitude', y='price',
                    marginal_x='histogram', marginal_y='histogram',
                    trendline='lowess', 
                    trendline_options=dict(frac=0.1),
                    trendline_color_override='yellow')
fig.show()

In [ ]:
fig = px.scatter(eda_df, x='longitude', y='price',
                    marginal_x='histogram', marginal_y='histogram',
                    trendline='lowess', 
                    trendline_options=dict(frac=0.1),
                    trendline_color_override='yellow')
fig.show()

## Create the output file

In [ ]:
merged_df.to_csv(path_to_use+output_filename)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cb0b277f-d226-41e6-8798-2eb04c8159dd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>